# Customizing IPython - Magics

IPython extends Python by adding shell-like commands called **magics**.

In [ ]:
%lsmagic

In [ ]:
import numpy

In [ ]:
%timeit A=numpy.random.random((1000,1000))

In [ ]:
%%timeit -n 1

A=numpy.random.random((1000,1000))
b = A.sum()


## Defining your own magic

As we have seen already, IPython has cell and line magics. You can define your own magics using any Python function and the `register_magic_function` method:

In [ ]:
ip = get_ipython()

In [ ]:
import time

def sleep_magic(line):
    """A simple function for sleeping"""
    t = float(line)
    time.sleep(t)

In [ ]:
ip.register_magic_function?

In [ ]:
ip.register_magic_function(sleep_magic, "line", "sleep")

In [ ]:
%sleep 2

In [ ]:
%sleep?

### Exercise

Define `%tic` and `%toc` magics, which can be use for simple timings, e.g. where

```python
for p in range(1,4):
    N = 10**p
    print "N=%i" % N
    %tic
    A = np.random.random((N,N))
    np.linalg.eigvals(A)
    %toc
```

each `%toc` will print the time since the last `%tic`. Create separate `tic` and `toc` functions that read and write
a global time variable.

In [ ]:
# %load soln/tictocf.py
import time

def format_time(dt):
    if dt < 1e-6:
        return u"%.3g ns" % (dt * 1e9)
    elif dt < 1e-3:
        return u"%.3g µs" % (dt * 1e6)
    elif dt < 1:
        return u"%.3g ms" % (dt * 1e3)
    else:
        return "%.3g s" % dt

def tic(line):
    global t0
    t0 = time.time()

def toc(line):
    global t0
    print (format_time(time.time() - t0))

ip = get_ipython()
ip.register_magic_function(tic)
ip.register_magic_function(toc)


In [ ]:
import numpy as np
import sys
for p in range(1,4):
    N = 10**p
    print("N=%i" % N)
    sys.stdout.flush()
    %tic
    A = np.random.random((N,N))
    np.linalg.eigvals(A)
    %toc

### Cell Magic

**Cell magics** take two args:

1. the **line** on the same line of the magic 
2. the **cell** the multiline body of the cell after the first line

In [ ]:
def dummy_cell_magic(line, cell):
    """dummy cell magic for displaying the line and cell it is passed"""
    print("line: %r" % line)
    print("cell: %r" % cell)

ip.register_magic_function(dummy_cell_magic, "cell", "dummy")

In [ ]:
%%dummy this is the line
this
is the
cell

In [ ]:
def parse_magic_line(line):
    """parse a magic line into a name and eval'd expression"""
    name, values_s = line.split(None, 1)
    values = eval(values_s, get_ipython().user_ns)
    return name, values

parse_magic_line("x range(5)")

#### Excercise

Can you write and register a **cell magic** that automates the outer iteration,
timing a block for various values of a particular variable:

In [ ]:
# %load soln/scalemagic.py
def scale_magic(line, cell):
    """run a cell block with a variety of input values"""
    name, values = parse_magic_line(line)
    ns = get_ipython().user_ns
    for v in values:
        assignment = "%s=%r" % (name, v)
        print(assignment)
        ns[name] = v
        sys.stdout.flush()
        %tic
        exec(cell, ns)
        %toc

ip = get_ipython()
ip.register_magic_function(scale_magic, "cell", "scale")


In [ ]:
%%scale N [ int(10**p) for p in range(1,4) ]

A = np.random.random((N,N))
np.linalg.eigvals(A)


In [ ]:
%%scale N [ int(2**p) for p in np.linspace(6, 11, 11) ]

A = np.random.random((N,N))
np.linalg.eigvals(A)


## Executing Notebooks

We can load a notebook into memory using `IPython.nbformat`.

In [ ]:
import io
import os

import nbformat as nbf

In [ ]:
def load_notebook(filename):
    """load a notebook object from a filename"""
    if not os.path.exists(filename) and not filename.endswith(".ipynb"):
        filename = filename + ".ipynb"
    with io.open(filename) as f:
        return nbf.read(f, as_version=4)


In [ ]:
nb = load_notebook("_Sample")

**A notebook is just a dictionary** with attribute access for convenience.

In [ ]:
nb.keys()

In [ ]:
cells = nb.cells
cells

We can see all the cells and their type

In [ ]:
for cell in cells:
    print()
    print('----- %s -----' % cell.cell_type)
    print(cell.source)

Now I can run all of the **code cells** with `get_ipython().run_cell`

In [ ]:
for cell in cells:
    ip = get_ipython()
    if cell.cell_type == 'code':
        ip.run_cell(cell.source, silent=True)

And we can now use the function that was defined in that notebook:

In [ ]:
nb_info(nb)

### Exercise

Can you write and register an `%nbrun` line magic to run a notebook?

```python
%nbrun Sample
```

In [ ]:
# %load soln/nbrun.py
def nbrun(line):
    """given a filename, execute the notebook in IPython"""
    nb = load_notebook(line)
    ip = get_ipython()
    for cell in nb.cells:
        if cell.cell_type == 'code':
            ip.run_cell(cell.source, silent=True)
    
get_ipython().register_magic_function(nbrun)


In [ ]:
%nbrun _Sample

The common way to make your magics reusable is to [write an Extension](Customizing%20IPython%20-%20Extensions.ipynb), so let's give that a try.